token_1.py is OK for milestone1

This file is still being improved...

In [75]:
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow
from tflearn.data_utils import VocabularyProcessor
import json
from sklearn.model_selection import train_test_split
from sklearn import metrics


enron_data_path = "../dataset/enron"

In [76]:
def preprocess_one_set(data_path):
    res = []
    for root, dirs, files in os.walk(data_path):
        for file_name in files:
            cur_file_path = os.path.join(root, file_name)
            cur_txt = ""
            with open(cur_file_path) as cur_f:
                try:
                    cur_lines = cur_f.readlines()
                    for one_line in cur_lines:
                        one_line = one_line.strip('\n')
                        one_line = one_line.strip('\r')
                        cur_txt += one_line
                    res.append(cur_txt)
                except UnicodeDecodeError:
                    continue
    return res

def process_all_data():
    spam_list = []
    ham_list = []
    NUM_SET = 1
    for i in range(1, NUM_SET + 1):
        spam_path = enron_data_path + str(i) +  "/spam/"
        ham_path = enron_data_path + str(i) + "/ham/"
        cur_spam_list = preprocess_one_set(spam_path)
        cur_ham_list = preprocess_one_set(ham_path)
        spam_list += cur_spam_list
        ham_list += cur_ham_list
    return spam_list, ham_list

In [77]:
spam_list, ham_list = process_all_data()
emails_list = spam_list + ham_list
len_spam = len(spam_list)
len_ham = len(ham_list)
print("len of spam_list: ", len_spam)
print("len of ham_list: ", len_ham)
y = [1] * len_spam + [0] * len_ham

len of spam_list:  1487
len of ham_list:  3672


In [78]:
def feature_extraction_bagofwords(emails_list, num_features):
    # currently do not consider max_features
    tfidfv = TfidfVectorizer(
        decode_error = "ignore",
        analyzer = "word",
        stop_words = "english",
        smooth_idf = False,
        max_features = num_features,# at most 2010 features for two datasets
    )
    x = np.array(emails_list)
    x = tfidfv.fit_transform(x)
    x = x.toarray()
    # vocab_path = "../output/vocabulary_bagofwords.txt"
    # print("output the vocabulary to " + vocab_path + " ......\n")
    # with open(vocab_path, 'w') as f:
    #     f.write(json.dumps(tfidfv.vocabulary_))
    print("len of x: ", len(x))
    print("#features: ",len(x[0]))
    return x


def feature_extraction_vo(emails_list):
    vp = VocabularyProcessor(
        max_document_length = 1000,
        min_frequency = 1,
        vocabulary = None,
        tokenizer_fn = None
    )
    x = vp.fit_transform(emails_list)
    x = np.array(list(x))
    vocab_path = "../output/vocabulary_tf.txt"
    # with open(vocab_path, 'w') as f:
    #     f.write(json.dumps(vp.vocabulary_._mapping))
    print("len of x: ", len(x))
    return x

Support Vector Machines Algorithm

In [79]:
from sklearn import svm

def detection_svm(X_train, y_train, X_test, y_test):
    print("---Support Vector Machines Algorithm---")
    svm_c = svm.SVC()
    svm_c.fit(X_train, y_train)
    y_pred = svm_c.predict(X_test)

    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1 = metrics.f1_score(y_test, y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    print("precision: ", precision)
    print("recall: ", recall)
    print("f1: ", f1)
    print("accuracy: ", accuracy)
    print("\n")

K-nearest Neighbor Algorithm (KNN)

In [80]:
from sklearn.neighbors import KNeighborsClassifier

def detection_knn(X_train, y_train, X_test, y_test):
    print("---K-nearest Neighbor Algorithm (KNN)---")
    knnc = KNeighborsClassifier()
    knnc.fit(X_train, y_train)
    y_pred = knnc.predict(X_test)

    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1 = metrics.f1_score(y_test, y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    print("")
    print("precision: ", precision)
    print("recall: ", recall)
    print("f1: ", f1)
    print("accuracy: ", accuracy)
    print("\n")

Gaussian Naive Bayes

In [81]:
from sklearn.naive_bayes import GaussianNB

def detection_gnb(X_train, y_train, X_test, y_test):
    print("---Gaussian Naive Bayes---")
    gnb = GaussianNB()
    gnb.fit(X_train, y_train)
    y_pred = gnb.predict(X_test)

    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1 = metrics.f1_score(y_test, y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    print("precision: ", precision)
    print("recall: ", recall)
    print("f1: ", f1)
    print("accuracy: ", accuracy)
    print("\n")

K-means Clustering

In [82]:
from sklearn.cluster import KMeans

def detection_kmeans(X_train, y_train, X_test, y_test, num_clusters):
    print("---K-means Clustering---")
    kmc = KMeans(n_clusters = num_clusters)
    kmc.fit(X_train)
    y_pred = kmc.predict(X_test)

    # print(y_test)
    # print(y_pred)
    y_processed = []
    ss = set()
    for yi in y_pred:
        ss.add(yi)
        if yi == 0:
            y_processed.append(0)
        else:
            y_processed.append(1)
    print("ss: ", ss)
    precision = metrics.precision_score(y_test, y_processed)
    recall = metrics.recall_score(y_test, y_processed)
    f1 = metrics.f1_score(y_test, y_processed)
    accuracy = metrics.accuracy_score(y_test, y_processed)
    print("precision: ", precision)
    print("recall: ", recall)
    print("f1: ", f1)
    print("accuracy: ", accuracy)
    print("\n")

Random Forest

In [83]:
from sklearn.ensemble import RandomForestClassifier

def detection_rf(X_train, y_train, X_test, y_test):
    print("---Random Forest---")
    rfc = RandomForestClassifier()
    rfc.fit(X_train, y_train)
    y_pred = rfc.predict(X_test)

    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1 = metrics.f1_score(y_test, y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    print("precision: ", precision)
    print("recall: ", recall)
    print("f1: ", f1)
    print("accuracy: ", accuracy)
    print("\n")

Training and Testing

Bag of Words

In [84]:
TEST_SIZE = 0.4
NUM_FEATURE = 5000

x = feature_extraction_bagofwords(emails_list, NUM_FEATURE)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, random_state=0)

len of x:  5159
#features:  5000


In [85]:
detection_svm(X_train, y_train, X_test, y_test)
detection_knn(X_train, y_train, X_test, y_test)
detection_gnb(X_train, y_train, X_test, y_test)
detection_kmeans(X_train, y_train, X_test, y_test, 3)
detection_rf(X_train, y_train, X_test, y_test)

---Support Vector Machines Algorithm---
precision:  0.9774696707105719
recall:  0.9724137931034482
f1:  0.9749351771823681
accuracy:  0.9859496124031008


---K-nearest Neighbor Algorithm (KNN)---

precision:  0.9730700179533214
recall:  0.9344827586206896
f1:  0.953386103781882
accuracy:  0.9743217054263565


---Gaussian Naive Bayes---
precision:  0.9376083188908145
recall:  0.9327586206896552
f1:  0.9351771823681937
accuracy:  0.9636627906976745


---K-means Clustering---
ss:  {0, 1, 2}
precision:  0.0
recall:  0.0
f1:  0.0
accuracy:  0.4748062015503876


---Random Forest---
precision:  0.9417637271214643
recall:  0.9758620689655172
f1:  0.9585097375105843
accuracy:  0.9762596899224806




VO

In [86]:
TEST_SIZE = 0.4

x = feature_extraction_vo(emails_list)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, random_state=0)

len of x:  5159


In [87]:
detection_svm(X_train, y_train, X_test, y_test)
detection_knn(X_train, y_train, X_test, y_test)
detection_gnb(X_train, y_train, X_test, y_test)
detection_kmeans(X_train, y_train, X_test, y_test, 3)
detection_rf(X_train, y_train, X_test, y_test)

---Support Vector Machines Algorithm---
precision:  0.8040201005025126
recall:  0.27586206896551724
f1:  0.41078305519897307
accuracy:  0.7776162790697675


---K-nearest Neighbor Algorithm (KNN)---

precision:  0.5408560311284046
recall:  0.2396551724137931
f1:  0.3321385902031064
accuracy:  0.7291666666666666


---Gaussian Naive Bayes---
precision:  0.4439252336448598
recall:  0.16379310344827586
f1:  0.23929471032745595
accuracy:  0.7073643410852714


---K-means Clustering---
ss:  {0, 1, 2}
precision:  0.251357220412595
recall:  0.7982758620689655
f1:  0.3823286540049546
accuracy:  0.2751937984496124


---Random Forest---
precision:  0.8793103448275862
recall:  0.4396551724137931
f1:  0.5862068965517242
accuracy:  0.8255813953488372


